In [12]:
from hardwareModel import HardwareModel
import networkx as nx
import cvxpy as cp
import numpy as np

In [13]:
hw = HardwareModel(cfg="aladdin_const_with_mem")

netlist: [('Add0', {'type': 'pe', 'function': 'Add', 'in_use': 0, 'idx': 0}), ('Regs0', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 0}), ('Regs1', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 1}), ('Regs2', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 2}), ('Mult0', {'type': 'pe', 'function': 'Mult', 'in_use': 0, 'idx': 0}), ('Eq0', {'type': 'pe', 'function': 'Eq', 'in_use': 0, 'idx': 0}), ('Buf0', {'type': 'mem', 'function': 'Buf', 'in_use': 0, 'idx': 0, 'size': 22}), ('Buf1', {'type': 'mem', 'function': 'Buf', 'in_use': 0, 'idx': 1, 'size': 22}), ('Mem0', {'type': 'mem', 'function': 'MainMem', 'in_use': 0, 'idx': 0, 'size': 1024}), ('Regs3', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 3})]


In [14]:
print(hw.netlist)
print(hw.area)

DiGraph with 10 nodes and 38 edges
{'Add': 379348800.0, 'Sub': 379348800.0, 'And': 50369960.0, 'BitAnd': 50369960.0, 'BitOr': 50369960.0, 'BitXor': 50369960.0, 'Eq': 379348800.0, 'FloorDiv': 8340007000.0, 'Gt': 379348800.0, 'GtE': 379348800.0, 'Invert': 50369960.0, 'IsNot': 50369960.0, 'LShift': 24964610.0, 'Lt': 379348800.0, 'LtE': 379348800.0, 'Mod': 379348800.0, 'Mult': 8340007000.0, 'Not': 50369960.0, 'NotEq': 379348800.0, 'Or': 50369960.0, 'RShift': 24964610.0, 'Regs': 5981433.0, 'UAdd': 379348800.0, 'USub': 379348800.0, 'Buf': 0.0, 'MainMem': 5981433.0}


In [15]:
hw.area['Add']

379348800.0

In [16]:
newGraph = nx.DiGraph()
newGraph.add_nodes_from([(1, {'function' : 'Mult'}), (2, {'function' : 'Add'})])
newGraph.add_edges_from([(1, 2)])

In [17]:
#digraph = hw.netlist
digraph = newGraph
nodes = digraph.nodes
edges = digraph.edges
#print("Nodes:", list(nodes))
#print("Edges:", list(edges))
digraph.nodes.data(True)
#digraph.nodes.data['function']

NodeDataView({1: {'function': 'Mult'}, 2: {'function': 'Add'}})

In [18]:
node_to_index = {node: i for i, node in enumerate(nodes)}
n = len(nodes) 
E = [(node_to_index[src], node_to_index[dst]) for src, dst in edges]

# Some fixed lij values for each edge (will slowly decrease the value until it doesn't work)
l = {edge: 10 for edge in E}

# Random weights for example, you may replace or adjust these
#w = np.random.randn(n) #fix this

w = np.asarray([[1] for _ in node_to_index])
W = cp.diag(w)
print(f'W matrix: {W}')
#w = np.abs(np.random.randn(n))
#w = list(hw.area.values())

p = cp.Variable( (n, 2) )


#objective = cp.Maximize(cp.quad_form(p, np.diag(w)))

# sum(diag(W(pp^T)) (Implement first)

#objective = cp.Maximize( w[0] * (cp.norm(p[0]) ** 2) ) 
# wDiag = cp.diag(w)
# product = p @ p.T
# objective = cp.Maximize(cp.sum(cp.multiply(wDiag, product)))

# pSquared = np.array([cp.norm(p[i]) for i in range(n)]) ** 2
# mult = cp.multiply(w, pSquared)
# objective.cp.Maximize(sum(mult))

# pPT = p @ p.T
# mult = w.T @ pPT
# diagonals = cp.diag(w.T @ p @ p.T)

#objective = cp.Minimize(-cp.sum(cp.diag(p @ p.T)))
objective = cp.Minimize(-sum([w[i] * (cp.norm(p[i]) ** 2) for i in range(len(w))]))

constraints = [cp.sum(cp.multiply(w, p)) == 0]  # Sum of w_i * p_i = 0

# Adding |p_i - p_j| <= l_ij constraints for each (i, j) in E
for (i, j) in E:
    constraints += [cp.abs(p[i] - p[j]) <= l[(i, j)]]

problem = cp.Problem(objective, constraints)
problem.solve()

W matrix: diag_vec(reshape([[1.00]
 [1.00]], (2,), F), 0)


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP, even though each sub-expression is.
You are trying to minimize a function that is concave.

In [42]:
def compute_Kij(i, j, n):
    # Create the unit vectors e_i and e_j
    e_i = np.zeros(n)
    e_j = np.zeros(n)
    e_i[i] = 1
    e_j[j] = 1
    
    # Compute the difference
    diff = e_i - e_j
    
    # Compute the outer product of the difference with itself
    Kij = np.outer(diff, diff)
    
    return Kij

In [43]:
print(nodes)

hw.area

[1, 2]


{'Add': 379348800.0,
 'Sub': 379348800.0,
 'And': 50369960.0,
 'BitAnd': 50369960.0,
 'BitOr': 50369960.0,
 'BitXor': 50369960.0,
 'Eq': 379348800.0,
 'FloorDiv': 8340007000.0,
 'Gt': 379348800.0,
 'GtE': 379348800.0,
 'Invert': 50369960.0,
 'IsNot': 50369960.0,
 'LShift': 24964610.0,
 'Lt': 379348800.0,
 'LtE': 379348800.0,
 'Mod': 379348800.0,
 'Mult': 8340007000.0,
 'Not': 50369960.0,
 'NotEq': 379348800.0,
 'Or': 50369960.0,
 'RShift': 24964610.0,
 'Regs': 5981433.0,
 'UAdd': 379348800.0,
 'USub': 379348800.0,
 'Buf': 0.0,
 'MainMem': 5981433.0}

In [44]:

node_to_index = {node: i for i, node in enumerate(nodes)}
#print(edges)
newEdges = []

for edge in edges:
    if (edge[1], edge[0]) in newEdges:
        continue

    newEdges.append(edge)

E = [(node_to_index[src], node_to_index[dst]) for src, dst in newEdges]
n = len(nodes)
w = np.asarray([[1.0] for _ in node_to_index])
areas = hw.area

for node in digraph.nodes.data(True):
    index = node_to_index[node[0]]
    w[index, 0] = areas[node[1]['function']]

diag_w = cp.diag(w)
wwT = w @ w.T
Ti = 100
di = 1
alpha = 1
Q = cp.Variable((n, n), PSD=True)
t = cp.Variable(n)

objective = cp.Maximize(cp.sum(cp.multiply(diag_w, Q))) 

constraints = [cp.sum(cp.multiply(wwT, Q)) == 0, Q >= 0, t >= 0] 

for i in range(n):
    constraints += [t[i] <= i * (Ti / n)]

#checkForDoubly = set()

for pair in E:
    #print(pair)
    first = pair[0]
    sec = pair[1]

    # if (pair[1], pair[0]) in checkForDoubly:
    #     continue

    # checkForDoubly.add(pair)
    constraints += [t[first] + di + alpha * cp.sum(cp.multiply(compute_Kij(first, sec, n), Q)) <= t[sec]]

prob = cp.Problem(objective, constraints)
prob.solve(solver='SCS', eps=1e-9, verbose=False)

print("The optimal value is", prob.value, '\n')
print("A solution t is", t.value, '\n')
print("A solution Q is", Q.value)

The optimal value is 3.783723471715324e-18 

A solution t is [-3.68848321e-15  1.87309873e+01] 

A solution Q is [[-5.68122203e-27  6.58651368e-26]
 [ 6.58651368e-26  1.34876280e-25]]


In [41]:
print(Q.value == Q.value.T)

[[ True  True]
 [ True  True]]


In [23]:
Q_matrix = Q.value

# for i in range(len(Q_matrix)):
#     for j in range(len(Q_matrix[0])):
#         if Q_matrix[i, j] < 0:
#             Q_matrix[i, j] = 0

#print(Q_matrix)

eigenvalues, eigenvectors = np.linalg.eigh(Q_matrix) 
print(eigenvalues)

# for i in range(len(eigenvalues)):
#     if eigenvalues[i] < 0:
#         eigenvalues[i] = 0

# D = np.diag(np.sqrt(eigenvalues))
# P = eigenvectors @ D

# print(Q.value)
# print('\n')
# print(P @ P.T)

[-3.17213118e-26  1.60916370e-25]


In [24]:
x1 = cp.Variable()
y1 = cp.Variable()
x2 = cp.Variable()
y2 = cp.Variable()
t1 = cp.Variable()
t2 = cp.Variable()

#w = np.array([1.0, 1.0]).reshape((2, 1))
a1 = 1.0
a2 = 1.0
T = 100.0
di = 1.0
alpha = 1.0
q1 = x1 ** 2 + y1 ** 2
q4 = x2 ** 2 + y2 ** 2
q2 = -( (a1 ** 2) * q1 + (a2 ** 2) * q4 ) / (2 * a1 * a2)
Q = np.array([ [q1, q2], [q2, q4] ])
KQ_inner_prod = q1 - 2 * q2 + q4

objective = cp.Maximize(a1 * q1 + a2 * q4)

constraints = [t1 + di + alpha * KQ_inner_prod <= T, Q >= 0]




Exception: Cannot evaluate the truth value of a constraint or chain constraints, e.g., 1 >= x >= 0.

In [25]:
# x1 = cp.Variable()
# y1 = cp.Variable()
# x2 = cp.Variable()
# y2 = cp.Variable()
t1 = cp.Variable()
t2 = cp.Variable()
p = cp.Variable((2, 2))

Q = p.T @ p
print(Q.shape)
q1 = Q[0, 0]
q2 = Q[0, 1]
q3 = Q[1, 0]
q4 = Q[1, 1]
KQ_inner_prod = q1 - 2 * q2 + q4
wwT_Q = q1 * (a1 ** 2) + q2 * a1 * a2 + q3 * a1 * a2 + q4 * (a2 ** 2)

a1 = 1.0
a2 = 1.0
T = 100.0
di = 1.0
alpha = 1.0

# q1 = x1 ** 2 + y1 ** 2
# q4 = x2 ** 2 + y2 ** 2
# q2 = -( (a1 ** 2) * q1 + (a2 ** 2) * q4 ) / (2 * a1 * a2)

print(Q)
objective = cp.Maximize(a1 * q1 + a2 * q4)

constraints = [t1 + di + alpha * KQ_inner_prod <= t2, Q >= 0, t1 <= T, t2 <= T, wwT_Q == 0]

prob = cp.Problem(objective, constraints)
prob.solve(verbose=True)

(2, 2)
var462.T @ var462
                                     CVXPY                                     
                                     v1.4.3                                    
(CVXPY) May 15 10:05:56 AM: Your problem has 6 variables, 5 constraints, and 0 parameters.
(CVXPY) May 15 10:05:56 AM: It is compliant with the following grammars: 
(CVXPY) May 15 10:05:56 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 15 10:05:56 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 15 10:05:56 AM: Your problem is compiled with the CPP canonicalization backend.


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
var462.T @ var462
var462.T @ var462

In [36]:
t1 = cp.Variable()
t2 = cp.Variable()
Q = cp.Variable((2, 2))
#p = cp.Variable((2, 2))

#Q = p.T @ p
print(Q.shape)
q1 = Q[0, 0]
q2 = Q[0, 1]
q3 = Q[1, 0]
q4 = Q[1, 1]
KQ_inner_prod = q1 - 2 * q2 + q4
wwT_Q = q1 * (a1 ** 2) + q2 * a1 * a2 + q3 * a1 * a2 + q4 * (a2 ** 2)

a1 = 1.0
a2 = 1.0
T = 100.0
di = 1.0
alpha = 1.0

# q1 = x1 ** 2 + y1 ** 2
# q4 = x2 ** 2 + y2 ** 2
# q2 = -( (a1 ** 2) * q1 + (a2 ** 2) * q4 ) / (2 * a1 * a2)

print(Q)
objective = cp.Maximize(a1 * q1 + a2 * q4)

constraints = [t1 + di + alpha * KQ_inner_prod <= t2, Q >= 0, t1 <= T, t2 <= T, wwT_Q == 0]

prob = cp.Problem(objective, constraints)
prob.solve(solver='SCS', verbose=False)
print(prob.value, '\n')
print(Q.value)
print('\n')

eigenvalues, eigenvectors = np.linalg.eigh(Q.value) 
print(eigenvalues)

(2, 2)
var837
5.683264772838868e-07 

[[ 2.84163239e-07 -4.54562813e-07]
 [-1.13551245e-07  2.84163239e-07]]


[1.70611994e-07 3.97714483e-07]


In [39]:
print(Q.value == Q.value.T)

[[ True False]
 [False  True]]


In [46]:
print(node_to_index)

{1: 0, 2: 1}


In [62]:
KQ_inner = cp.sum(cp.multiply(compute_Kij(0, 1, 2), Q.value))
length_dict = {}

for edge in newEdges:
    first = edge[0]
    sec = edge[1]
    length = np.sum(np.multiply(compute_Kij(node_to_index[first], node_to_index[sec], len(node_to_index)), Q.value)) ** 0.5

    if edge[0] not in length_dict:
        length_dict[first] = {}

    length_dict[first][sec] = length

print(length_dict)

{1: {2: nan}}


/var/folders/35/z588hjp90z92x4_wc7v7bytc0000gn/T/ipykernel_25652/2220452440.py:7: RuntimeWarning: invalid value encountered in scalar power
  length = np.sum(np.multiply(compute_Kij(node_to_index[first], node_to_index[sec], len(node_to_index)), Q.value)) ** 0.5
